In [1]:
import os, sys, json, time, re
import random, collections, cPickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle
import spacy
sys.path.append('/Users/matt.meng/dev/seq2seq_model/')
%matplotlib inline  

In [2]:
data_path = '/Users/matt.meng'
file_name = 'insights_article_data_title_only_20170719_20170728.json'
meta_data_file_name = 'meta_title_data.csv'
output_pickle_file = 'new_processed_titles_data.pkl'
delimiter = '\t\t'

In [3]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def convert_text_JSON_to_csv(data_path, csv_path, delimiter='\t'):
    start_time = time.time()
    batch_size = 2000
    raw_content = ""
    expected_keys = ['title', 'url', 'traffic', 'publisherId']
    with open(data_path, 'r') as raw_input:
        with open(csv_path, 'w') as raw_output:
            counter = 0
            raw_output.write(delimiter.join(expected_keys) + "\n")
            for line in raw_input:
                json_doc = json.loads(line)
                if not all([key in json_doc.keys() for key in expected_keys]):
                    continue
                elem_list = [json_doc['title'], json_doc['url'], json_doc['traffic'], json_doc['publisherId']]
                if not all([isEnglish(elem) for elem in elem_list]):
                    continue
                raw_content += delimiter.join(elem_list) + "\n"
                counter += 1
                if counter % batch_size == 0:
                    raw_output.write(raw_content)
                    raw_content = ""
                if counter % (20 * batch_size) == 0:
                    print 'finished processing {} rows using {:.2f} seconds'.format(counter, time.time() - start_time)
    print 'finished processing all the data using {:.2f} seconds'.format(time.time() - start_time)

In [4]:
convert_text_JSON_to_csv(os.path.join(data_path, file_name), os.path.join(data_path, meta_data_file_name), delimiter)

finished processing 40000 rows using 4.64 seconds
finished processing 80000 rows using 8.99 seconds
finished processing 120000 rows using 12.94 seconds
finished processing 160000 rows using 18.03 seconds
finished processing 200000 rows using 22.68 seconds
finished processing 240000 rows using 25.94 seconds
finished processing 280000 rows using 29.19 seconds
finished processing 320000 rows using 32.48 seconds
finished processing 360000 rows using 35.76 seconds
finished processing 400000 rows using 38.99 seconds
finished processing 440000 rows using 42.23 seconds
finished processing 480000 rows using 45.47 seconds
finished processing 520000 rows using 48.68 seconds
finished processing 560000 rows using 52.18 seconds
finished processing 600000 rows using 56.85 seconds
finished processing 640000 rows using 61.46 seconds
finished processing 680000 rows using 65.20 seconds
finished processing all the data using 66.85 seconds


In [5]:
data = pd.read_csv(os.path.join(data_path, meta_data_file_name), index_col='url', delimiter=delimiter, encoding='utf-8')

/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [7]:
print data.shape
data.head()

(698007, 3)


,title,traffic,publisherId
url,,,
http://pix11.com/2017/07/17/iconic-lower-east-side-cup-and-saucer-diner-closes-its-doors-for-food-after-rent-nearly-doubles/,Iconic Lower East Side Cup and Saucer diner cl...,240.0,1002700.0
http://www.cbsnews.com/news/ulysha-renee-hall-first-female-chief-dallas-police-department/?ftag=COS-05-10aaa0h&utm_campaign=trueAnthem: Trending Content&utm_content=5970682c00bd470007264f17&utm_medium=trueAnthem&utm_source=facebook,Ulysha Hall makes history as Dallas Police Dep...,116.0,1038583.0
http://www.abante-tonite.com/student-lumaklak-ng-asido.htm,UP STUDENT LUMAKLAK NG ASIDO! - Abante Tonite,384.0,1062055.0
http://www.jsonline.com/story/news/2017/06/17/paul-nehlen-challenge-paul-ryan-after-losing-68-points-last-year/405597001/,Paul Nehlen to challenge Paul Ryan after losin...,352.0,1053378.0
http://www.msn.com/en-us/lifestyle/family-relationships/what-is-it-like-to-be-in-an-open-marriage-1-woman-shares-her-story/ar-BBC6STn,What Is It Like to Be in an Open Marriage? 1 W...,219.0,1023406.0


In [8]:
url = 'https://www.cnet.com/news/pubg-playerunknown-brendan-greene-interview-july-2017/'
data.loc[url]

title          Finding PlayerUnknown, the my
traffic                                 3895
publisherId                      1.03858e+06
Name: https://www.cnet.com/news/pubg-playerunknown-brendan-greene-interview-july-2017/, dtype: object

In [9]:
data.dropna(how='any', inplace=True)
data['publisherId'] = data['publisherId'].astype(int).astype(str)
valid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',
                        '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',
                        '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',
                        '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',
                        '1021578', '1043813', '1010748', '1040526', '1005092', '612',
                        '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',
                        '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',
                        '1037842', '1029984', '723', '196', '1030941']

filtered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]
unique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]
print unique_filtered_data.shape

(128155, 3)


In [10]:
print unique_filtered_data.shape
unique_filtered_data.head()

(128155, 3)


,title,traffic,publisherId
url,,,
http://www.cbsnews.com/news/ulysha-renee-hall-first-female-chief-dallas-police-department/?ftag=COS-05-10aaa0h&utm_campaign=trueAnthem: Trending Content&utm_content=5970682c00bd470007264f17&utm_medium=trueAnthem&utm_source=facebook,Ulysha Hall makes history as Dallas Police Dep...,116.0,1038583
http://www.msn.com/en-us/lifestyle/family-relationships/what-is-it-like-to-be-in-an-open-marriage-1-woman-shares-her-story/ar-BBC6STn,What Is It Like to Be in an Open Marriage? 1 W...,219.0,1023406
https://www.msn.com/en-us/foodanddrink/recipes/add-excitement-to-your-scrambled-eggs-with-one-clever-ingredient/ar-AAos3oX,Add Excitement to Your Scrambled Eggs with One...,114.0,1068057
http://www.msn.com/en-us/movies/celebrity/valerians-dane-dehaan-says-his-love-for-new-baby-led-to-his-car-being-stolen/ar-AAot0bP,Valerian's Dane DeHaan Says His Love for New B...,113.0,1023406
http://www.ajc.com/news/health-med-fit-science/will-low-calorie-sugar-substitutes-artificial-sweeteners-actually-help-you-lose-weight/RsYQtTQo8BIE5HFbswtOTI/,"Will low-calorie sugar substitutes, artificial...",384.0,1041479


In [11]:
nlp = spacy.load('en')

In [41]:
index = 40
title_content = data['title'][index].decode('ascii')
doc1 = nlp(title_content)
print 'the content: ', title_content
print '-------------'
for ent in doc1.ents:
    print ent, ent.label_
print '-------------'
for token in doc1:
    print token, token.pos_, token.tag_, token.lemma_, token.is_stop

the content:  Tulane's goal in Year 2 under Willie Fritz: 'We want to get in a bowl game'
-------------
Tulane ORG
Year 2 DATE
Willie Fritz PERSON
-------------
Tulane PROPN NNP tulane False
's PART POS 's False
goal NOUN NN goal False
in ADP IN in True
Year PROPN NNP year False
2 NUM CD 2 False
under ADP IN under True
Willie PROPN NNP willie False
Fritz PROPN NNP fritz False
: PUNCT : : False
' PUNCT `` ' False
We PRON PRP -PRON- True
want VERB VBP want False
to PART TO to True
get VERB VB get True
in ADP IN in True
a DET DT a True
bowl NOUN NN bowl False
game NOUN NN game False
' PUNCT '' ' False


#### customize the processing functions

In [13]:

def basic_tokenizer(line, normalize_digits=True):
    line = line.replace("'s", '')
    line = re.sub(r"\'ve", " have ", line)
    line = re.sub(r"can't", "can not ", line)
    line = re.sub(r"n't", " not ", line)
    line = re.sub(r"I'm", "I am", line)
    line = re.sub(r" m ", " am ", line)
    line = re.sub(r"\'re", " are ", line)
    line = re.sub(r"\'d", " would ", line)
    line = re.sub(r"\'ll", " will ", line)
    line = re.sub(r"-", " ? ", line)
    #line = re.sub(r"!", " ! ", line)
    #line = re.sub(r":", " : ", line)
    line = re.sub('[\.,;\?"#%\'()*+/;<=>@\[\]^_{|}~\\\]', ' ', line)
    line = re.sub('[\n\t ]+', ' ', line)
    words = []
    _DIGIT_RE = re.compile(r"\d")
    for token in line.strip().lower().split():
        if not token:
            continue
        if normalize_digits:
            token = re.sub(_DIGIT_RE, b'#', token)
        words.append(token)
    return len(words), ' '.join(words)


def tokenize_title_column(data, processed_column_name, pageView_column_name='pageView', title_column_name='title'):
    data['title_word_counts'], data[processed_column_name] = zip(*data[title_column_name].map(basic_tokenizer))
    # sort by the title word counts and filter them
    sorted_data = data.sort_values(by=['title_word_counts', pageView_column_name], ascending=[True, False])
    index = (sorted_data['title_word_counts'] >= 5) & (sorted_data['title_word_counts'] <= 15)
    filtered_data = sorted_data.loc[index, :]
    print 'finish the tokenization...'
    return filtered_data


_PAD = b"_PAD"
_GO = b"_GO"
_EOS = b"_EOS"
_UNK = b"_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

TOKEN_DICT = {}
REVERSE_TOKEN_DICT = {}
for i in xrange(len(_START_VOCAB)):
    TOKEN_DICT[_START_VOCAB[i]] = i
    REVERSE_TOKEN_DICT[i] = _START_VOCAB[i]


def create_vocab_dict(data, column_name, pageView_column_name='pageView', token_freq_threshold=5, UKN_frac_threshold=0.3):
    vocab_dict = {}
    all_titles = []
    selected_titles = []
    selected_title_urls = []
    selected_title_pageView = []
    for title, url, pageView in zip(data[column_name], data.index, data[pageView_column_name]):
        words = []
        for token in title.split(' '):
            words.append(token)
            if token not in vocab_dict:
                vocab_dict[token] = 0
            vocab_dict[token] += 1
        all_titles.append((words, url, pageView))
    print 'total {} tokens are identified...'.format(len(vocab_dict))

    token_dict, reverse_token_dict = TOKEN_DICT.copy(), REVERSE_TOKEN_DICT.copy()
    UKN_index = len(token_dict) - 1
    unique_counts = 0
    sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
    for i, pair in enumerate(sorted_pairs):
        if pair[1] >= token_freq_threshold:
            unique_counts += 1
            token_dict[pair[0]] = i + 1 + UKN_index
            reverse_token_dict[(i + 1 + UKN_index)] = pair[0]
        else:
            token_dict[pair[0]] = UKN_index
    print 'total {} unique tokens are included in the token dictionary...'.format(unique_counts)

    for i in xrange(len(all_titles)):
        indexed_title = map(token_dict.get, all_titles[i][0])
        UKN_count = sum([elem == UKN_index for elem in indexed_title])
        if (1. * UKN_count / len(indexed_title)) < UKN_frac_threshold:
            selected_titles.append(indexed_title)
            selected_title_urls.append(all_titles[i][1])
            selected_title_pageView.append(all_titles[i][2])

    print 'total {} titles are included...'.format(len(selected_titles))
    return token_dict, reverse_token_dict, selected_titles, selected_title_urls, selected_title_pageView


In [14]:
title = 'those pointless upper-middle-class entitlements'
basic_tokenizer(title)

(6, 'those pointless upper middle class entitlements')

In [15]:
processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
filtered_data = tokenize_title_column(unique_filtered_data, processed_column_name, pageView_column_name)

/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


finish the tokenization...


In [16]:
print filtered_data.shape
filtered_data.head()

(119206, 5)


,title,traffic,publisherId,title_word_counts,processed_title
url,,,,,
http://www.msn.com/en-us/news/breakingnews/live-coverage-from-cbs-news/ar-BBmYvYY,Live Coverage from CBS News,348010.0,1023406,5,live coverage from cbs news
http://www.politico.com/story/2017/07/28/how-mccain-upended-obamacare-repeal-241070,How McCain upended Obamacare repeal,187020.0,612,5,how mccain upended obamacare repeal
http://www.huffingtonpost.com/katie-hurley/theres-nothing-selfish-about-suicide_b_5672519.html,There's Nothing Selfish About Suicide,104351.0,1040522,5,there nothing selfish about suicide
http://www.nbcnews.com/politics/white-house/can-president-pardon-himself-n785181,Can the president pardon himself?,95782.0,1010748,5,can the president pardon himself
http://www.huffingtonpost.com/entry/why-im-a-racist_us_57893b9ee4b0e7c873500382,Why I'm A Racist,81273.0,1040522,5,why i am a racist


In [ ]:
#titles = filtered_data['processed_title'].tolist()

In [17]:
index = 15
title_content = filtered_data['processed_title'][index]
title_content = title_content.decode('ascii')
doc = nlp(title_content)
print title_content
print '-------------'
for token in doc:
    print token, token.pos_, token.tag_, token.lemma_, token.is_stop
print '-------------'
for token in doc:
    if (token.pos_ == u'NOUN' or token.pos_ == u'PROPN') and not token.is_stop:
        print token
print '-------------'
for ent in doc.ents:
    print ent, ent.label_

doctor who actor passes away
-------------
doctor NOUN NN doctor False
who NOUN WP who True
actor NOUN NN actor False
passes VERB VBZ pass False
away ADV RB away False
-------------
doctor
actor
-------------


In [18]:
stop_words = set(['msn', 'breitbart', 'news', 'commentary', 'coverage'])

def process_title_column(data, title_column_name, pageView_column_name):
    '''function to create the vocabulary dictionary and collect
    the titles according to the selection rules (include only the nourns.)
    '''
    all_titles, vocab_dict = [], {}
    count, start_time = 0, time.time()
    nlp = spacy.load('en')
    for title, url, pageView in zip(data[title_column_name], data.index, data[pageView_column_name]):
        words = []
        title_content = title.decode('ascii')
        doc = nlp(title_content)
        count += 1
        if count % 10000 == 0:
            print 'finish {} using {:.2f} seconds'.format(count, time.time() - start_time)
        if count == 50000:
            break
            
        for token in doc:
            word = token.lemma_.encode('ascii')
            if word in stop_words or '#' in word:
                continue
            if (token.pos_ == u'NOUN' or token.pos_ == u'PROPN') and not token.is_stop:
                if word not in words:
                    # the title is restricted to contain only only entities
                    # and exlude the duplicate words
                    words.append(word) 
                if word not in vocab_dict:
                    vocab_dict[word] = 0
                vocab_dict[word] += 1
        if len(words) > 0:
            all_titles.append((words, url, pageView))
    print 'total {} tokens are identified...'.format(len(vocab_dict))
    return all_titles, vocab_dict



_PAD = b"_PAD"
_GO = b"_GO"
_EOS = b"_EOS"
_UNK = b"_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

TOKEN_DICT = {}
REVERSE_TOKEN_DICT = {}
for i in xrange(len(_START_VOCAB)):
    TOKEN_DICT[_START_VOCAB[i]] = i
    REVERSE_TOKEN_DICT[i] = _START_VOCAB[i]
    

def create_selected_vocab_dict(vocab_dict, UKN_index, token_freq_threshold):
    '''process the vocabulary dictionary and use `token_freq_threshold`
    to create a token dictionary
    '''
    token_dict, reverse_token_dict = TOKEN_DICT.copy(), REVERSE_TOKEN_DICT.copy()
    unique_counts = 0
    sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
    for i, pair in enumerate(sorted_pairs):
        if pair[1] >= token_freq_threshold:
            unique_counts += 1
            token_dict[pair[0]] = i + 1 + UKN_index
            reverse_token_dict[(i + 1 + UKN_index)] = pair[0]
        else:
            token_dict[pair[0]] = UKN_index
    print 'total {} unique tokens are included in the token dictionary...'.format(unique_counts)
    return token_dict, reverse_token_dict


def process_title_with_token_dict(all_titles, token_dict, UKN_index, UKN_frac_threshold):
    selected_titles = []
    selected_title_urls = []
    selected_title_pageView = []
    for i in xrange(len(all_titles)):
        indexed_title = map(token_dict.get, all_titles[i][0])
        if len(indexed_title) == 0:
            continue
        UKN_count = sum([elem == UKN_index for elem in indexed_title])
        if (1. * UKN_count / len(indexed_title)) < UKN_frac_threshold:
            selected_titles.append(indexed_title)
            selected_title_urls.append(all_titles[i][1])
            selected_title_pageView.append(all_titles[i][2])

    print 'total {} titles are included...'.format(len(selected_titles))
    #return token_dict, reverse_token_dict, selected_titles, selected_title_urls, selected_title_pageView
    return {'url': selected_title_urls,
            'titles': selected_titles,
            'pageViw': selected_title_pageView,
            'token_dict': token_dict,
            'reverse_token_dict': reverse_token_dict}

In [19]:
UKN_index = len(TOKEN_DICT) - 1

In [20]:
all_titles, vocab_dict = process_title_column(filtered_data, 'processed_title', 'traffic')

finish 10000 using 10.08 seconds
finish 20000 using 16.01 seconds
finish 30000 using 21.87 seconds
finish 40000 using 29.02 seconds
finish 50000 using 36.54 seconds
total 14623 tokens are identified...


In [21]:
print len(all_titles)

49854


#### check the topK entities

In [22]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
for i in xrange(40):
    print sorted_pairs[i]

('trump', 2612)
('man', 1641)
('woman', 1284)
('police', 1257)
('year', 828)
('horoscope', 766)
('death', 660)
('health', 650)
('house', 632)
('report', 586)
('home', 572)
('city', 531)
('state', 516)
('thing', 506)
('video', 504)
('photo', 497)
('crash', 480)
('world', 475)
('bill', 465)
('star', 461)
('car', 459)
('baby', 430)
('school', 422)
('russia', 419)
('love', 417)
('john', 408)
('child', 407)
('care', 407)
('time', 403)
('family', 399)
('park', 395)
('way', 388)
('people', 383)
('cancer', 383)
('session', 368)
('fire', 368)
('county', 366)
('life', 365)
('game', 363)
('mccain', 361)


#### creat the token dictionary

In [23]:
token_dict, reverse_token_dict = create_selected_vocab_dict(vocab_dict, UKN_index, token_freq_threshold=4)

total 6330 unique tokens are included in the token dictionary...


#### creat the content dictionary

In [24]:
processed_content = process_title_with_token_dict(all_titles, token_dict, UKN_index, UKN_frac_threshold=0.3)

total 44571 titles are included...


In [32]:
processed_content.keys()
content = processed_content.copy()

In [33]:
def print_content_by_index(content, index):
    reverse_token_dict = content['reverse_token_dict']
    print content['pageViw'][index]
    print content['url'][index]
    print map(reverse_token_dict.get, content['titles'][index])
    print '-----------------------'

In [34]:
for index in xrange(30 ,50):
    print_content_by_index(processed_content, index)

15244.0
http://www.huffingtonpost.com/entry/are-all-inclusive-resorts-safe_us_59722b30e4b0f1feb89b42fe
['resort']
-----------------------
14655.0
http://www.msn.com/en-us/news/politics/al-gore-?i-was-wrong?-on-trump/ar-BBEG7Ik
['politic', 'al', 'gore']
-----------------------
14532.0
http://www.msnbc.com/am-joy/watch/advice-to-trump-staffers-run-949699651614
['advice', 'staffer']
-----------------------
14455.0
http://www.businessinsider.com/13-chains-taking-over-america-2017-7
['chain', 'america']
-----------------------
13134.0
http://www.politico.com/story/2017/07/25/house-overwhelmingly-approves-russia-sanctions-240951
['house', 'russia', 'sanction']
-----------------------
12752.0
http://www.fox29.com/news/269835700-story
['company', 'employee', 'microchip', 'implant']
-----------------------
12619.0
http://dailymail.co.uk/news/article-4737052
['west', 'dog', 'diamond']
-----------------------
12554.0
http://www.msn.com/en-us/money/companies/what-happened-when-walmart-left/ar-BBEk

In [44]:
benchmark_index = 80
print_content_by_index(content, benchmark_index)

5751.0
http://www.dailymail.co.uk/femail/article-4714974/Celebrity-Beach-Vacation-Beauty-Looks.html
['celebrity', 'beauty']
-----------------------


In [36]:
print benchmark_index

70


In [45]:
threshold = 0.5
benchmark_title_len = len(content['titles'][benchmark_index])
for index in xrange(len(content['titles'])):
    common_counts = 0
    for word_index in content['titles'][benchmark_index]:
        if word_index in content['titles'][index]:
            common_counts += 1
    max_len = max(len(content['titles'][index]), benchmark_title_len)
    if 1.*common_counts/max_len >= threshold:
        print_content_by_index(content, index)

5751.0
http://www.dailymail.co.uk/femail/article-4714974/Celebrity-Beach-Vacation-Beauty-Looks.html
['celebrity', 'beauty']
-----------------------
5429.0
http://dailymail.co.uk/news/article-4714974
['celebrity', 'beauty']
-----------------------
1150.0
http://dailym.ai/2vovLaU
['celebrity', 'beauty']
-----------------------
875.0
http://www.huffingtonpost.com/2013/10/16/gorgeous-celebrities-over-60_n_4109868.html
['celebrity']
-----------------------
127.0
http://metro.co.uk/2013/04/06/lindsay-lohan-jennifer-aniston-and-mel-bs-boobs-real-or-fake-3564290/
['celebrity', 'boob']
-----------------------
63.0
http://www.wsbtv.com/news/national/hot-topics/weird-first-tweets-of-celebrities/560204554
['tweet', 'celebrity']
-----------------------
54.0
http://www.independent.co.uk/life-style/fashion/15-best-online-beauty-retailers-beauty-pie-cult-beauty-space-nk-birchbox-9158731.html
['beauty', 'retailer']
-----------------------
3011.0
http://www.huffingtonpost.com/vanessa-cunningham/dangerou

#### check the titles by keyword

In [38]:
processed_content['titles'][30]

[931]

In [39]:
def search_title_by_keyword(key_word, content, token_dict, reverse_token_dict):
    count = 0
    token_index = token_dict[key_word]
    for index in xrange(len(processed_content['titles'])):
        if token_index in content['titles'][index]:
            count += 1
            print_content_by_index(content, index)
    print 'found total {} relevant titles'.format(count)

In [47]:
search_title_by_keyword('transgender', content, token_dict, reverse_token_dict)

8910.0
https://a.msn.com/r/2/AAoSqjT?m=en-us
['opinion', 'trump', 'transgender', 'patriot']
-----------------------
5835.0
http://www.huffingtonpost.com/brynn-tannehill/the-truth-about-transgend_b_8564834.html
['truth', 'transgender', 'suicide']
-----------------------
1767.0
https://www.usatoday.com/story/news/2017/07/26/transgender-troops-not-under-trump/511726001/
['transgender', 'troop', 'trump']
-----------------------
307.0
http://www.nydailynews.com/news/national/transgender-navy-seal-article-1.1362490
['transgender', 'navy', 'seal']
-----------------------
181.0
http://www.nbcnews.com/feature/nbc-out/pakistan-issues-landmark-transgender-passport-n778076
['issue', 'transgender', 'passport']
-----------------------
159.0
https://www.usatoday.com/story/news/politics/2013/07/23/transsexuals-military-struggle-acceptance/2513153/
['transgender', 'troop', 'silence']
-----------------------
66.0
http://www.huffingtonpost.com/joanne-herman/lgbt-rights-mental-illnes_b_605595.html
['illne

In [ ]:
plt.hist(counter.values(), bins=100)
#plt.yscale('log')
axes = plt.gca()
axes.set_xlim([0,20])
#axes.set_ylim([ymin,ymax])

In [ ]:
data_path = '/Users/matt.meng/Downloads'
file_name = 'small_articles.json'

In [ ]:
#bad_titles

In [ ]:
pickle_file = 'processed_title_data.pkl'

content = {'url' : selected_title_urls, 
           'titles' : titles, 
           'pageViw' : selected_title_pageView,
           'token_dict' : token_dict,
           'reverse_token_dict' : reverse_token_dict}

with open(pickle_file, 'wb') as handle:
    cPickle.dump(content, handle, protocol=cPickle.HIGHEST_PROTOCOL)